In [ ]:
!pip install statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
import os
from datetime import datetime

In [ ]:
# Read Yahoo Finance Dataset
collated_df = pd.read_csv('..\data\meta\Yahoo Finance.csv')
# Convert Earnings Date and Previous Earnings Date to datetime format
collated_df['Earnings Date'] = pd.to_datetime(collated_df['Earnings Date'], format = '%d/%m/%Y')
collated_df['Previous Earnings Date'] = pd.to_datetime(collated_df['Previous Earnings Date'], format = '%d/%m/%Y')
# Number of days between each quarter
collated_df['Delta'] = (collated_df['Earnings Date'] - collated_df['Previous Earnings Date']).dt.days
sentiment_df = collated_df[['Ticker', 'Earnings Date', 'Previous Earnings Date', 'Delta']].copy(deep = True)
sentiment_lst = sentiment_df.values.tolist()
# Initialise columns for Reddit sentiment analysis
collated_df['Reddit Skew'] = np.nan
collated_df['Reddit Kurtosis'] = np.nan
# Initialise columns for Twitter sentiment analysis
collated_df['Twitter Sentiment'] = np.nan
collated_df['Twitter SD'] = np.nan
collated_df['Twitter Skew'] = np.nan
collated_df['Twitter Kurtosis'] = np.nan
# Initialise column for combined distribution analysis
collated_df['Social Media Covariance'] = np.nan

In [ ]:
collated_df

,Earnings Date,EPS Estimate,Reported EPS,Surprise(%),Previous Earnings Date,Price Change,Coefficient Variation of Stock Price,Percentage Change from prev reported EPS,EPS Beats,Ticker,Sector,Delta,Reddit_Sentiment,Reddit_Polarity
0,2022-09-07,-0.38,-0.35,0.0741,2022-06-01,-0.122655,0.106351,-0.269231,1,GME,Consumer Cyclical,98,NaN,NaN
1,2022-06-01,-0.36,-0.52,-0.4325,2022-03-17,0.579668,0.219666,-0.217391,0,GME,Consumer Cyclical,76,NaN,NaN
2,2022-03-17,0.21,-0.46,-3.2249,2021-12-08,-0.535234,0.180643,-1.600000,0,GME,Consumer Cyclical,99,NaN,NaN
3,2021-12-08,-0.13,-0.35,-1.6977,2021-09-08,-0.160201,0.086445,-0.315789,0,GME,Consumer Cyclical,91,NaN,NaN
4,2021-09-08,-0.17,-0.19,-0.1446,2021-06-09,-0.324167,0.164528,0.545455,0,GME,Consumer Cyclical,91,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,2021-02-11,1.46,1.47,0.0096,2020-10-01,0.014390,0.025595,-0.120482,1,PEP,Consumer Defensive,133,NaN,NaN
8,2020-10-01,1.49,1.66,0.1156,2020-07-13,0.023988,0.016441,0.128788,1,PEP,Consumer Defensive,80,NaN,NaN
9,2020-07-13,1.25,1.32,0.0543,2020-04-28,-0.005354,0.012858,0.168224,1,PEP,Consumer Defensive,76,NaN,NaN
10,2020-04-28,1.03,1.07,0.0398,2020-02-13,-0.074019,0.081405,-0.289655,1,PEP,Consumer Defensive,75,NaN,NaN


In [ ]:
sentiment_lst

[['GME',
  Timestamp('2022-09-07 00:00:00'),
  Timestamp('2022-06-01 00:00:00'),
  98],
 ['GME',
  Timestamp('2022-06-01 00:00:00'),
  Timestamp('2022-03-17 00:00:00'),
  76],
 ['GME',
  Timestamp('2022-03-17 00:00:00'),
  Timestamp('2021-12-08 00:00:00'),
  99],
 ['GME',
  Timestamp('2021-12-08 00:00:00'),
  Timestamp('2021-09-08 00:00:00'),
  91],
 ['GME',
  Timestamp('2021-09-08 00:00:00'),
  Timestamp('2021-06-09 00:00:00'),
  91],
 ['GME',
  Timestamp('2021-06-09 00:00:00'),
  Timestamp('2021-03-23 00:00:00'),
  78],
 ['GME',
  Timestamp('2021-03-23 00:00:00'),
  Timestamp('2020-12-08 00:00:00'),
  105],
 ['GME',
  Timestamp('2020-12-08 00:00:00'),
  Timestamp('2020-09-09 00:00:00'),
  90],
 ['GME',
  Timestamp('2020-09-09 00:00:00'),
  Timestamp('2020-06-09 00:00:00'),
  92],
 ['GME',
  Timestamp('2020-06-09 00:00:00'),
  Timestamp('2020-03-26 00:00:00'),
  75],
 ['GME',
  Timestamp('2020-03-26 00:00:00'),
  Timestamp('2019-12-10 00:00:00'),
  107],
 ['GME',
  Timestamp('2019-12-

# Twitter Sentiment Score Collation

In [ ]:
# Change Directory for Twitter Data
os.chdir('../data/datafiles/')

In [ ]:
# Initialize a list to collect files not found
fnf = []

# Loop through each stock per quarter in the Yahoo Finance list
for idx, ticker in enumerate(sentiment_lst):
    # Format the file name
    file = f'twitter/{ticker[0]}_Tweets_{ticker[4]}.csv'
    print(f'Initiating {idx}, {ticker[0]} {ticker[4]} collation...')

    try:
        # Read the CSV file if it exists
        ticker_df = pd.read_csv(file)

        # Initialize a datetime series from the previous earnings date to the current earnings date
        tmp = np.zeros(ticker[3]+1)
        index = pd.date_range(start=ticker[2], end=ticker[1], freq='D')
        data = pd.Series(tmp, index)

        # Convert date column to datetime format
        ticker_df['date'] = pd.to_datetime(ticker_df['date'], format = '%Y-%m-%d')
        ticker_df['collated_score'] = np.nan

        # Calculcate a collated sentiment score for each date in the range
        # This score is a weighted average based on retweet count + 1 (to include the Tweet author in the count)
        # Retweet count + 1 serves as a measure of the number of people who agree with the sentiment
        for date in index:
            mask = ticker_df['date'] == date
            ticker_df.loc[mask,'collated_score'] = ticker_df.loc[mask, 'compound_score'] * ((ticker_df.loc[mask,'retweets_count']+1)/(ticker_df.loc[mask,'retweets_count'].sum()+ticker_df[mask].count()[0]))
            data[date] = ticker_df.loc[mask, 'collated_score'].sum()

        # Perform a linear interpolation on the data such that the distribution is not affected
        data.interpolate(inplace=True)

        # Perform exponential smoothing with alpha = 0.2 on the data
        # Heuristic initialization method is only available if the number of observations is more than 10
        try:
            model_fit = SimpleExpSmoothing(data, initialization_method="heuristic").fit(
                smoothing_level=0.2, optimized=False
            )
        except:
            model_fit = SimpleExpSmoothing(data).fit(
                smoothing_level=0.2, optimized=False
            )
        
        # Obtain the final value as the aggregated sentiment score over the whole period
        model_fcast = model_fit.forecast(1)
        
        # Input the final aggregated sentiment score into the df to be saved
        collated_df.loc[idx, 'Twitter Sentiment'] = model_fcast[-1]
        print(f'Completed {idx}, {ticker[0]} {ticker[4]} collation.')
    
    # If file not found, append to fnf list
    except FileNotFoundError:
        print(f'File Not Found: {idx}, {file}')
        fnf.append((idx, ticker))

In [ ]:
# Check for files not found - Should have only 1 observation
print(len(fnf), fnf)

In [ ]:
# Check that there is only one missing value corresponding to the file not found
collated_df['Twitter Sentiment'].isna().sum()

# Get Distribution for Reddit & Twitter Posts

In [ ]:
# Initialize a dictionary for Reddit and a list for Twitter
count, fnf = {}, []

# Loop through each stock per quarter in the Yahoo Finance list
for idx, ticker in enumerate(sentiment_lst):
    # Format the file names for Twitter and Reddit
    file_t = f'twitter/{ticker[0]}_Tweets_{ticker[4]}.csv'
    file_r = f'reddit/{ticker[0]}_posts.csv'
    print(f'Initiating {idx}, {ticker[0]} {ticker[4]} collation...')

    try:
        # Read the CSV files for Twitter and Reddit if they exist
        twitter_df = pd.read_csv(file_t)
        reddit_df = pd.read_csv(file_r, index_col = 0)

        # Calculate standard deviation, skewness and kurtosis for Twitter sentiment scores
        twitter_sd = twitter_df['compound_score'].std()
        collated_df.loc[idx, 'Twitter SD'] = twitter_sd

        twitter_skew = twitter_df['compound_score'].skew()
        collated_df.loc[idx, 'Twitter Skew'] = twitter_skew

        twitter_kurtosis = twitter_df['compound_score'].kurtosis()
        collated_df.loc[idx, 'Twitter Kurtosis'] = twitter_kurtosis

        # Get the Reddit posts for this quarter only
        if ticker[0] not in count:
            count[ticker[0]] = reddit_df['quarter'].max()
        else:
            count[ticker[0]] -= 1
        reddit_df = reddit_df.loc[reddit_df['quarter'] == count[ticker[0]]].copy(deep = True)

        # Calculate the skewness and kurtosis for Reddit sentiment scores
        reddit_skew = reddit_df['weighted_score'].skew()
        collated_df.loc[idx, 'Reddit Skew'] = reddit_skew

        reddit_kurtosis = reddit_df['weighted_score'].kurtosis()
        collated_df.loc[idx, 'Reddit Kurtosis'] = reddit_kurtosis

        # Initialize two datetime series for Twitter and Reddit from the previous earnings date to the current earnings date
        tmp = np.zeros(ticker[3])
        index = pd.date_range(start=ticker[2], end=ticker[1], freq='D')
        data_t = pd.Series(tmp, index[1:])
        data_r = pd.Series(tmp, index[1:])

        # Convert date columns to datetime format
        twitter_df['date'] = pd.to_datetime(twitter_df['date'], format = '%Y-%m-%d')
        twitter_df['collated_score'] = np.nan

        reddit_df['date'] = reddit_df['created_utc'].apply(lambda timestamp : datetime.fromtimestamp(timestamp).strftime('%d-%m-%y'))
        reddit_df['date'] = pd.to_datetime(reddit_df['date'], format = '%d-%m-%y')
        reddit_df['collated_score'] = np.nan

        # Calculate the weighted average for both Twitter and Reddit sentiment scores
        for date in index[:0:-1]:
            mask_t = twitter_df['date'] == date
            twitter_df.loc[mask_t,'collated_score'] = twitter_df.loc[mask_t, 'compound_score'] * ((twitter_df.loc[mask_t,'retweets_count']+1)/(twitter_df.loc[mask_t,'retweets_count'].sum()+twitter_df[mask_t].count()[0]))
            data_t[date] = twitter_df.loc[mask_t, 'collated_score'].sum()

            mask_r = reddit_df['date'] == date
            reddit_df.loc[mask_r,'collated_score'] = reddit_df.loc[mask_r, 'weighted_score'] * (reddit_df.loc[mask_r,'score']/reddit_df.loc[mask_r,'score'].sum())
            data_r[date] = reddit_df.loc[mask_r, 'collated_score'].sum()

        # Perform linear interpolation on the data such that the distribution is not affected
        data_t.interpolate(inplace=True)
        data_r.interpolate(inplace=True)

        # Calculate the covariance between Twitter and Reddit daily average sentiment scores
        # Input the aggregated social media covariance into the df to be saved
        if not reddit_df.empty:
            collated_cov = data_t.cov(data_r)
            collated_df.loc[idx, 'Social Media Covariance'] = collated_cov
        
        print(f'Completed {idx}, {ticker[0]} {ticker[4]} collation.')
    
    # Account for file not found error
    except FileNotFoundError:
        print(f'File Not Found: {idx}, {file_t}')
        # fnf.append((idx, ticker))

In [ ]:
# Check the number of missing values for each distribution statistic
collated_df[['Twitter SD', 'Twitter Skew', 'Twitter Kurtosis', 'Social Media Covariance']].isna().sum()

In [ ]:
# Drop the redundant column
collated_df = collated_df.drop('Earnings Date STR', axis=1)

In [ ]:
# Uncomment this line if you wish to save to CSV
# collated_df.to_csv('Twitter_Sentiments.csv', index=False)